In [ ]:
import math
import yaml
import numpy as np
from scipy.stats import lognorm
from scipy.optimize import minimize_scalar
from scipy.optimize import brentq
import matplotlib.pyplot as plt
from scipy.stats import kstest
from tools_util import readModelInputs, getFacDict
from pyrheautils import pathTranslate

In [ ]:
runDesc = '/home/welling/git/pyRHEA_github/src/sim/week_run_OC.yaml'

inputDict = readModelInputs(runDesc)
facDict = getFacDict(inputDict)


In [ ]:
fac = 'UCIM'
assert facDict[fac]['category'] == 'HOSPITAL', '%s is not a hospital' % fac
print facDict[fac]['losModel']
hospLM = facDict[fac]['losModel']
fracPatientDaysICU = facDict[fac]['fracAdultPatientDaysICU']['value']
print facDict[fac]['meanLOSICU']['value']


In [ ]:
with open(pathTranslate('$(MODELDIR)/constants/hospital_constants.yaml'), 'rU') as f:
    hospConstants = yaml.load(f)
print hospConstants['icuLOSLogNormSigma']

In [ ]:
icuLOSLogNormSigma = hospConstants['icuLOSLogNormSigma']['value']
meanLOSICU = facDict[fac]['meanLOSICU']['value']
mu = math.log(meanLOSICU) - (0.5 * icuLOSLogNormSigma * icuLOSLogNormSigma)
icuLM = {'parms': [mu, icuLOSLogNormSigma], 'pdf': 'lognorm(mu=$0,sigma=$1)'}
print icuLM

In [ ]:
hospCRV = lognorm(hospLM['parms'][1], scale=math.exp(hospLM['parms'][0]))
icuCRV = lognorm(icuLM['parms'][1], scale=math.exp(icuLM['parms'][0]))
print hospCRV.mean()
print icuCRV.mean()

## Derivation of meanLOSHosp ##

$fracPatientDaysHosp=1.0-fracPatientDaysICU$

$(A+B)*meanLOSHosp+B*meanLOSICU=(A+B)*meanLOS$

$(B*meanLOSICU)/((A+B)*meanLOS) = fracPatientDaysICU$

$(B*meanLOSICU)/fracPatientDaysICU = (A+B)*meanLOS$

$meanLOSICU/(fracPatientDaysICU*meanLOS) = (A+B)/B$

$(fracPatientDaysICU*meanLOS)/(meanLOSICU) = B/(A+B)$

$meanLOSHosp + (B/(A+B))*meanLOSICU = meanLOS$

$meanLOSHosp = meanLOS - (B/(A+B))*meanLOSICU$

In [ ]:
fracICU = (fracPatientDaysICU * hospCRV.mean())/icuCRV.mean()
print fracICU
meanLOSHosp = hospCRV.mean() - (fracICU*icuCRV.mean())
print meanLOSHosp

In [ ]:
def crvFromMeanAndLogNormSigma(meanV, logNormSigma):
    mu = math.log(meanV) - (0.5 * logNormSigma * logNormSigma)
    return lognorm(logNormSigma, scale=math.exp(mu))



In [ ]:
def jointSamps(nSamps, aCRV, bCRV, frac):
    aSamps = aCRV.rvs(nSamps)
    bSamps = bCRV.rvs(nSamps)
    choices = np.random.choice(2, nSamps, p=[frac, 1.0 - frac])
    return np.choose(choices, [aSamps + bSamps, aSamps])

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
plt.rcParams["figure.figsize"] = [16, 4.5]

def plotit(samples, sampLbl, outerCRV, outerCRVLbl, titleStr):
    nBins = 200
    histoRange = [0.0, 20.0]
    nSamps = len(samples)
    scale = len(samples) * (histoRange[1] - histoRange[0])/nBins
    plt.hist(samples, bins=nBins, range=histoRange, label=sampLbl)
    xV = np.linspace(histoRange[0], histoRange[1], nBins)
    yV = scale * outerCRV.pdf(xV)
    plt.plot(xV, yV, '-', label=outerCRVLbl)
    plt.legend()
    plt.title(titleStr)
    plt.show()



In [ ]:
nSamps = 1000000
plotit(jointSamps(nSamps, hospCRV, icuCRV, fracICU), 'joint ICU + HOSP', 
       hospCRV, 'expected', 'Uncorrected')

## Correction based on stdv matching ##

In [ ]:
def func(val, nSamps, meanLOSHosp, hospCRV, icuCRV):
    guessCRV = crvFromMeanAndLogNormSigma(meanLOSHosp, val)
    samps = jointSamps(nSamps, guessCRV, icuCRV, fracICU)
    #print 'mean: %s vs %s' % (np.mean(samps), hospCRV.mean())
    #print 'stdv: %s vs %s' % (np.std(samps), hospCRV.std())
    return np.std(samps) - hospCRV.std()

rslt = brentq(func, 0.1 * hospLM['parms'][1], hospLM['parms'][1], maxiter=10,
             disp=False, args=(10000, meanLOSHosp, hospCRV, icuCRV))
print rslt
stdvCorrCRV = crvFromMeanAndLogNormSigma(meanLOSHosp, rslt)
nSamps = 1000000
plotit(jointSamps(nSamps, stdvCorrCRV, icuCRV, fracICU), 'joint ICU + HOSP', 
       hospCRV, 'expected', 'Corrected based on stdv')

## Corrected via K-S matching ##

In [ ]:
def func(val, nSamps, meanLOSHosp, hospCRV, icuCRV):
    guessCRV = crvFromMeanAndLogNormSigma(meanLOSHosp, val)
    samps = jointSamps(nSamps, guessCRV, icuCRV, fracICU)
    rslt = kstest(samps, hospCRV.cdf).statistic
    #print '%s -> %s' % (val, rslt)
    return rslt

rslt = minimize_scalar(func, [0.1 * hospLM['parms'][1], hospLM['parms'][1]],
                      args=(1000000, meanLOSHosp, hospCRV, icuCRV))
print rslt
ksCorrCRV = crvFromMeanAndLogNormSigma(meanLOSHosp, rslt.x)
nSamps = 1000000
uncorSamps = jointSamps(nSamps, hospCRV, icuCRV, fracICU)
uncorMean = np.mean(uncorSamps)
plotit(uncorSamps, 'joint ICU + HOSP', 
       hospCRV, 'expected', '%s Uncorrected (mean %f)' % (fac, uncorMean))
corSamps = jointSamps(nSamps, ksCorrCRV, icuCRV, fracICU)
corMean = np.mean(corSamps)
plotit(corSamps, 'joint ICU + HOSP', 
       hospCRV, 'expected', '%s Corrected based on K-S fit (mean %f)' % (fac, corMean))